In [2]:
import pandas as pd
import numpy as np
import altair as alt
#from scipy import stats

In [3]:
survey_data = pd.read_csv("../data/Raw/kaggle_survey_2020_responses.csv").loc[:,:"Q24"]
survey_data.shape

C:\Users\jacob\miniconda3\envs\532\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(20037, 119)

* Q3: residence country (query)
* Q4: highest level of formal education aquired. (tile plot)
* Q5: current or most recent job title (query).
* Q6: Programming years. (query)
* Q7: Programming Languages Used on a regular basis. (bar plot)
* Q8: Programming Languages Recommended to learn first for Aspiring Data Scientist (tile plot)
* Q17: Machine learning algorithms used on a regular basis. (barplot)
+ Q24: Yearly Compensation (USD) (card)



In [4]:
#survey_data.columns.isin(important_questions)
stage_1_data = survey_data[survey_data.columns[survey_data.columns.str.contains("Q[3-8]|Q17|Q24",regex=True)]]

In [6]:
# The instances where code has never been written before are filtered.
remove_phrase = ["I have never written code"]
stage_1_data = stage_1_data[~stage_1_data["Q6"].isin(remove_phrase)]

In [7]:
# income wrangeling

stage_2_data = stage_1_data.query('Q24 != "$0-999" & Q24 != "1,000-1,999" & Q24 != "2,000-2,999" & Q24 != "3,000-3,999" & Q24 != "4,000-4,999" & Q24 != "5,000-5,999"')
stage_2_data = stage_2_data.iloc[1:]
stage_2_data['Q24'] = stage_2_data['Q24'].replace("> $500,000", "500,000-600,000")
stage_2_data['Q24'] = stage_2_data['Q24'].replace("$500,000", "500,000-600,000")
stage_2_data['Q24'] = stage_2_data['Q24'].replace(",", "", regex=True)
stage_2_data[['lower','higher']] = stage_2_data.Q24.str.split("-", expand=True)
stage_2_data["lower"] = pd.to_numeric(stage_2_data["lower"])
stage_2_data["higher"] = pd.to_numeric(stage_2_data["higher"])
stage_2_data.query("Q5 == 'Statistician' &  Q6 == '10-20 years'")["lower"].median()
#stage_2_data["higher"].median()
stage_2_data.head()

,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,Q7_Part_4,Q7_Part_5,Q7_Part_6,...,Q17_Part_6,Q17_Part_7,Q17_Part_8,Q17_Part_9,Q17_Part_10,Q17_Part_11,Q17_OTHER,Q24,lower,higher
1,Colombia,Doctoral degree,Student,5-10 years,Python,R,SQL,C,NaN,NaN,...,"Dense Neural Networks (MLPs, etc)",Convolutional Neural Networks,NaN,Recurrent Neural Networks,NaN,NaN,NaN,NaN,NaN,NaN
2,United States of America,Master’s degree,Data Engineer,5-10 years,Python,R,SQL,NaN,NaN,NaN,...,NaN,Convolutional Neural Networks,NaN,NaN,"Transformer Networks (BERT, gpt-3, etc)",NaN,NaN,100000-124999,100000.0,124999.0
3,Argentina,Bachelor’s degree,Software Engineer,10-20 years,NaN,NaN,NaN,NaN,NaN,Java,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15000-19999,15000.0,19999.0
4,United States of America,Master’s degree,Data Scientist,5-10 years,Python,NaN,SQL,NaN,NaN,NaN,...,"Dense Neural Networks (MLPs, etc)",NaN,NaN,NaN,NaN,NaN,NaN,125000-149999,125000.0,149999.0
5,Japan,Master’s degree,Software Engineer,3-5 years,Python,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
stage_2_data.to_csv("..\data\Processed\general_processed_data.csv", index = False)

In [9]:
stage_2_data.to_csv('general_processed_data.csv', index = False)

# Language Barplot wrangling 

In [15]:
language_barplot_survey_data = stage_1_data.iloc[1:].filter(regex = 'Q[3-7]', axis =1)
language_columns = language_barplot_survey_data.columns[language_barplot_survey_data.columns.str.contains("Q7", regex=True)].tolist()
id_columns = language_barplot_survey_data.columns[language_barplot_survey_data.columns.str.contains("Q3|Q4|Q5|Q6", regex=True)].tolist()
lang_barplot_data = language_barplot_survey_data.melt(id_vars= id_columns, value_vars=language_columns, var_name="prog_lang", value_name="selected_lang").dropna()

In [16]:
lang_barplot_data

,Q3,Q4,Q5,Q6,prog_lang,selected_lang
0,Colombia,Doctoral degree,Student,5-10 years,Q7_Part_1,Python
1,United States of America,Master’s degree,Data Engineer,5-10 years,Q7_Part_1,Python
3,United States of America,Master’s degree,Data Scientist,5-10 years,Q7_Part_1,Python
4,Japan,Master’s degree,Software Engineer,3-5 years,Q7_Part_1,Python
5,India,Bachelor’s degree,Data Analyst,< 1 years,Q7_Part_1,Python
...,...,...,...,...,...,...
245803,Ireland,Some college/university study without earning ...,Software Engineer,1-2 years,Q7_OTHER,Other
245815,Poland,Master’s degree,Data Analyst,5-10 years,Q7_OTHER,Other
245825,Netherlands,Master’s degree,Data Scientist,10-20 years,Q7_OTHER,Other
245829,Other,Professional degree,Other,3-5 years,Q7_OTHER,Other


In [17]:
lang_barplot_data.to_csv('../data/Processed/lang_barplot_data.csv', index = False)

# ML Methods Barplot wrangling 

In [19]:
machine_learning_barplot_survey_data = stage_1_data.iloc[1:].filter(regex = 'Q[3456]|Q17', axis =1)
ml_columns = machine_learning_barplot_survey_data.columns[machine_learning_barplot_survey_data.columns.str.contains("Q17", regex=True)].tolist()
id_columns = machine_learning_barplot_survey_data.columns[machine_learning_barplot_survey_data.columns.str.contains("Q3|Q4|Q5|Q6", regex=True)].tolist()
ml_barplot_data = machine_learning_barplot_survey_data.melt(id_vars= id_columns, value_vars=ml_columns, var_name="ml_method", value_name="selected_ml_method").dropna()


# Porting over Yuyan's Shortened Names to General Data Wrangling
ml_barplot_data = ml_barplot_data.replace({'Transformer Networks (BERT, gpt-3, etc)' : 'Transformer Networks',
                                          "Dense Neural Networks (MLPs, etc)" : "Dense Neural Networks",
                                          "Gradient Boosting Machines (xgboost, lightgbm, etc)" : "Gradient Boosting Machines",
                                          "None" : "I do not use ML Methods"
                                          })

In [20]:
ml_barplot_data

,Q3,Q4,Q5,Q6,ml_method,selected_ml_method
1,United States of America,Master’s degree,Data Engineer,5-10 years,Q17_Part_1,Linear or Logistic Regression
3,United States of America,Master’s degree,Data Scientist,5-10 years,Q17_Part_1,Linear or Logistic Regression
6,Brazil,Bachelor’s degree,Student,3-5 years,Q17_Part_1,Linear or Logistic Regression
11,United States of America,Doctoral degree,Research Scientist,1-2 years,Q17_Part_1,Linear or Logistic Regression
13,Canada,Bachelor’s degree,Data Engineer,< 1 years,Q17_Part_1,Linear or Logistic Regression
...,...,...,...,...,...,...
226689,Netherlands,Master’s degree,Data Scientist,10-20 years,Q17_OTHER,Other
226803,United States of America,Doctoral degree,Research Scientist,20+ years,Q17_OTHER,Other
226816,India,Bachelor’s degree,Student,1-2 years,Q17_OTHER,Other
226880,Morocco,Professional degree,Data Scientist,3-5 years,Q17_OTHER,Other


In [21]:
ml_barplot_data.to_csv('../data/Processed/ml_barplot_data.csv', index = False)

## Recommended First Programming Language vs Education Level Stacket bars or Fluctuation Plot

In [23]:
fluctuation_plot_survey_data = stage_1_data.iloc[1:].filter(regex = 'Q[3|4|5|6|8]', axis =1)
fluctuation_plot_survey_data

,Q3,Q4,Q5,Q6,Q8
1,Colombia,Doctoral degree,Student,5-10 years,Python
2,United States of America,Master’s degree,Data Engineer,5-10 years,Python
3,Argentina,Bachelor’s degree,Software Engineer,10-20 years,R
4,United States of America,Master’s degree,Data Scientist,5-10 years,Python
5,Japan,Master’s degree,Software Engineer,3-5 years,Python
...,...,...,...,...,...
20032,Turkey,Some college/university study without earning ...,NaN,NaN,NaN
20033,United Kingdom of Great Britain and Northern I...,Master’s degree,Currently not employed,20+ years,Python
20034,Brazil,Master’s degree,Research Scientist,< 1 years,Python
20035,India,Bachelor’s degree,Software Engineer,3-5 years,Python


In [24]:
fluctuation_plot_survey_data.to_csv('../data/Processed/Fluctuations_plot_data.csv', index = False)

In [25]:
fluctuation_plot_survey_data["Q8"].value_counts()

Python        14241
R              1259
SQL             849
C++             325
C               301
MATLAB          195
Java            167
Other           151
Julia           121
Javascript       88
None             81
Bash             26
Swift            17
Name: Q8, dtype: int64